## Upload list of California emergency rooms

Link: https://data.chhs.ca.gov/dataset/licensed-healthcare-facility-listing

(Download current list)

In [1]:
import pandas as pd

In [6]:
hf=pd.read_csv("current-healthcare-facility-listing.csv")

In [7]:
erlist=hf[hf["ER_SERVICE_LEVEL_DESC"].str.lower().str.contains("emergency")]

In [8]:
erlist.tail()

,OSHPD_ID,FACILITY_NAME,LICENSE_NUM,FACILITY_LEVEL_DESC,DBA_ADDRESS1,DBA_CITY,DBA_ZIP_CODE,COUNTY_CODE,COUNTY_NAME,ER_SERVICE_LEVEL_DESC,TOTAL_NUMBER_BEDS,FACILITY_STATUS_DESC,FACILITY_STATUS_DATE,LICENSE_TYPE_DESC,LICENSE_CATEGORY_DESC,LATITUDE,LONGITUDE
504,106560525,ADVENTIST HEALTH SIMI VALLEY,050000216,Parent Facility,2975 Sycamore Dr,Simi Valley,93065,56,Ventura,Emergency - Basic,144,Open,1965-07-29,Hospital,General Acute Care Hospital,34.289730,-118.743940
505,106560529,ST. JOHN'S REGIONAL MEDICAL CENTER,050000064,Parent Facility,1600 N ROSE AVE,OXNARD,93030,56,Ventura,Emergency - Basic,269,Open,1953-01-01,Hospital,General Acute Care Hospital,34.216870,-119.158190
508,106571086,WOODLAND MEMORIAL HOSPITAL,030000115,Parent Facility,1325 COTTONWOOD ST,WOODLAND,95695,57,Yolo,Emergency - Basic,105,Open,1967-03-29,Hospital,General Acute Care Hospital,38.662840,-121.793300
509,106574010,SUTTER DAVIS HOSPITAL,030000124,Parent Facility,2000 Sutter Pl,Davis,95616,57,Yolo,Emergency - Basic,48,Open,1994-09-24,Hospital,General Acute Care Hospital,38.562890,-121.771640
510,106580996,ADVENTIST HEALTH AND RIDEOUT,230000126,Parent Facility,726 FOURTH ST,MARYSVILLE,95901,58,Yuba,Emergency - Basic,221,Open,1946-06-05,Hospital,General Acute Care Hospital,39.138805,-121.593602


In [9]:
print("Number of emergency rooms found: %d" % len(erlist))

Number of emergency rooms found: 334


In [10]:
erlist["ER_SERVICE_LEVEL_DESC"].unique().tolist()

['Emergency - Basic', 'Emergency - Standby', 'Emergency - Comprehensive']

In [11]:
erlist["FACILITY_LEVEL_DESC"].unique().tolist()

['Parent Facility', 'Consolidated Facility']

In [18]:
import sqlalchemy as sal
import getpass

In [19]:
endpoint = "capstone.clihskgj8i7s.us-west-2.rds.amazonaws.com"
user="group3"
db="db1"
pw=getpass.getpass("Enter database password")

Enter database password········


In [20]:
engine = sal.create_engine('postgresql://%s:%s@%s/%s' % (user, pw, endpoint, db))

In [22]:
pd.read_sql("select * from hospitals limit 1", engine)

,oshpd_id,zipcode,name,county_name,latitude,longitude


In [24]:
df=erlist[["OSHPD_ID","DBA_ZIP_CODE","FACILITY_NAME","COUNTY_NAME","LATITUDE","LONGITUDE"]]

In [28]:
cols=df.columns.str.lower().tolist()
cols[1] = "zipcode"
cols[2] = "name"
df.columns=cols
df.set_index("oshpd_id", drop=True, inplace=True)

In [31]:
df.to_sql("hospitals", engine, if_exists="append")

In [34]:
pd.read_sql("select * from hospitals limit 10", engine, index_col="oshpd_id")

,zipcode,name,county_name,latitude,longitude
oshpd_id,,,,,
106010735,94501,ALAMEDA HOSPITAL,Alameda,37.76266,-122.253991
106010739,94705,ALTA BATES SUMMIT MEDICAL CENTER-ALTA BATES CA...,Alameda,37.85645,-122.257430
106010776,94609,UCSF BENIOFF CHILDREN'S HOSPITAL OAKLAND,Alameda,37.83722,-122.267470
106010846,94602,HIGHLAND HOSPITAL,Alameda,37.79925,-122.231380
106010937,94609,ALTA BATES SUMMIT MEDICAL CENTER,Alameda,37.82106,-122.262570
106010967,94545,ST. ROSE HOSPITAL,Alameda,37.63291,-122.089620
106010987,94538,WASHINGTON HOSPITAL - FREMONT,Alameda,37.55847,-121.980060
106013619,94578,SAN LEANDRO HOSPITAL,Alameda,37.71364,-122.140670
106014050,94588,STANFORD HEALTH CARE - VALLEYCARE,Alameda,37.69206,-121.880950
